In [ ]:
!pip install diffusers transformers accelerate torch torchvision
!pip install gradio safetensors

In [ ]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Stable Diffusion Text-to-Image Generator with Gradio UI
# Optimized for Google Colab with Real-time Progress Tracking

import os
import gc
import time
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, EulerDiscreteScheduler
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

class StableDiffusionGenerator:
    def __init__(self):
        self.pipe = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.progress_callback = None
        self.start_time = None
        self.total_steps = 0
        print(f"Using device: {self.device}")

    def progress_fn(self, step, timestep, latents):
        """Callback function to track generation progress"""
        if self.progress_callback and self.start_time:
            elapsed_time = time.time() - self.start_time
            progress_percent = (step / self.total_steps) * 100

            if step > 0:  # Avoid division by zero
                estimated_total_time = elapsed_time * self.total_steps / step
                remaining_time = estimated_total_time - elapsed_time
            else:
                remaining_time = 0

            # Format time nicely
            def format_time(seconds):
                if seconds < 60:
                    return f"{seconds:.1f}s"
                elif seconds < 3600:
                    return f"{seconds//60:.0f}m {seconds%60:.0f}s"
                else:
                    return f"{seconds//3600:.0f}h {(seconds%3600)//60:.0f}m"

            progress_text = f"🎨 Generating: {progress_percent:.1f}% complete | "
            progress_text += f"⏱️ Elapsed: {format_time(elapsed_time)} | "
            progress_text += f"⏳ Remaining: {format_time(remaining_time)}"

            # Update progress through callback
            self.progress_callback(progress_text)

    def load_model(self, model_id="runwayml/stable-diffusion-v1-5", progress=gr.Progress()):
        """Load the Stable Diffusion model with memory optimizations and progress tracking"""
        try:
            # Progress tracking for model loading
            progress(0.1, desc="🔄 Clearing previous models...")

            # Clear any existing model from memory
            if self.pipe is not None:
                del self.pipe
                torch.cuda.empty_cache()
                gc.collect()

            progress(0.3, desc="📥 Downloading model (this may take a while for first time)...")
            print(f"Loading model: {model_id}")

            # Load model with memory optimizations
            self.pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                use_safetensors=True,
                variant="fp16" if self.device == "cuda" else None,
                safety_checker=None,  # Disable safety checker to save memory
                requires_safety_checker=False
            )

            progress(0.7, desc="⚙️ Optimizing model for Colab...")

            # Use faster, more memory-efficient scheduler
            self.pipe.scheduler = EulerDiscreteScheduler.from_config(
                self.pipe.scheduler.config
            )

            # Enable all memory optimizations
            if self.device == "cuda":
                self.pipe.enable_attention_slicing(1)  # More aggressive slicing
                self.pipe.enable_vae_slicing()
                self.pipe.enable_vae_tiling()  # Additional VAE optimization

            progress(0.9, desc="🚀 Moving model to device...")
            self.pipe = self.pipe.to(self.device)

            # Enable CPU offloading to save VRAM
            if self.device == "cuda":
                self.pipe.enable_model_cpu_offload()

            progress(1.0, desc="✅ Model loaded successfully!")

            # Clear cache after loading
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            print("Model loaded successfully!")
            return "✅ Model loaded successfully! Ready to generate images."

        except Exception as e:
            error_msg = f"❌ Error loading model: {str(e)}"
            print(error_msg)
            return error_msg

    def generate_image(self, prompt, negative_prompt="", num_inference_steps=15,
                      guidance_scale=7.5, width=512, height=512, seed=-1,
                      progress=gr.Progress()):
        """Generate image from text prompt with real-time progress"""

        if self.pipe is None:
            return None, "❌ Please load a model first!"

        try:
            # Validate inputs for optimal Colab performance
            if width > 768 or height > 768:
                return None, "❌ Image dimensions too large for Colab! Keep width and height ≤ 768px"

            if num_inference_steps > 30:
                return None, "❌ Too many inference steps! Keep steps ≤ 30 for faster generation"

            # Set up progress tracking
            self.total_steps = num_inference_steps
            self.start_time = time.time()

            # Progress callback for Gradio
            def update_progress(text):
                progress(None, desc=text)

            self.progress_callback = update_progress

            progress(0.05, desc="🎲 Setting up generation parameters...")

            # Set random seed if specified
            if seed != -1:
                torch.manual_seed(seed)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed(seed)

            progress(0.1, desc="🧹 Clearing memory cache...")

            # Clear cache before generation
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            progress(0.15, desc=f"🎨 Starting image generation ({num_inference_steps} steps)...")
            print(f"Generating image for prompt: '{prompt}'")

            # Generate image with progress callback
            with torch.autocast(self.device):
                result = self.pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    width=width,
                    height=height,
                    callback=self.progress_fn,
                    callback_steps=1  # Update progress every step
                )

            image = result.images[0]

            progress(0.95, desc="🖼️ Finalizing image...")

            # Clear cache after generation
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            total_time = time.time() - self.start_time
            status_msg = f"✅ Image generated successfully in {total_time:.1f}s!"
            progress(1.0, desc=status_msg)
            print(status_msg)
            return image, status_msg

        except Exception as e:
            error_msg = f"❌ Error generating image: {str(e)}"
            print(error_msg)
            progress(1.0, desc=error_msg)
            return None, error_msg

    def get_memory_info(self):
        """Get current memory usage information"""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**3
            reserved = torch.cuda.memory_reserved() / 1024**3
            total = torch.cuda.get_device_properties(0).total_memory / 1024**3
            return f"GPU Memory - Used: {allocated:.2f}GB | Reserved: {reserved:.2f}GB | Total: {total:.1f}GB"
        else:
            return "Running on CPU"

# Initialize the generator
generator = StableDiffusionGenerator()

# Define the Gradio interface
def create_interface():
    with gr.Blocks(title="Fast Stable Diffusion Generator", theme=gr.themes.Soft()) as interface:

        gr.Markdown("# ⚡ Fast Stable Diffusion Text-to-Image Generator")
        gr.Markdown("**Optimized for Google Colab with Real-time Progress Tracking**")

        with gr.Tab("🎨 Image Generation"):
            with gr.Row():
                with gr.Column(scale=1):
                    # Model loading section
                    gr.Markdown("### 1. Load Model")
                    with gr.Row():
                        model_dropdown = gr.Dropdown(
                            choices=[
                                "runwayml/stable-diffusion-v1-5",  # Fastest, most stable
                                "stabilityai/stable-diffusion-2-1-base",  # Good balance
                                "CompVis/stable-diffusion-v1-4"  # Alternative option
                            ],
                            value="runwayml/stable-diffusion-v1-5",
                            label="Select Model (v1-5 is fastest)"
                        )
                        load_btn = gr.Button("🚀 Load Model", variant="primary", size="sm")

                    load_status = gr.Textbox(label="Model Status", interactive=False)

                    gr.Markdown("### 2. Generation Settings")
                    prompt = gr.Textbox(
                        label="✨ Prompt",
                        placeholder="a beautiful sunset over mountains, highly detailed, cinematic lighting",
                        lines=3
                    )
                    negative_prompt = gr.Textbox(
                        label="🚫 Negative Prompt (Optional)",
                        placeholder="blurry, bad quality, distorted, ugly",
                        lines=2
                    )

                    with gr.Row():
                        steps = gr.Slider(
                            minimum=8, maximum=30, value=15, step=1,
                            label="⚡ Inference Steps (15 recommended)",
                            info="Fewer steps = faster generation"
                        )
                        guidance = gr.Slider(
                            minimum=3, maximum=15, value=7.5, step=0.5,
                            label="🎯 Guidance Scale",
                            info="Higher = follows prompt more closely"
                        )

                    with gr.Row():
                        width = gr.Dropdown(
                            choices=[256, 384, 512, 640, 768],
                            value=512,
                            label="📐 Width (px)"
                        )
                        height = gr.Dropdown(
                            choices=[256, 384, 512, 640, 768],
                            value=512,
                            label="📐 Height (px)"
                        )

                    seed = gr.Number(
                        label="🎲 Seed (-1 for random)",
                        value=-1,
                        precision=0,
                        info="Same seed = same image"
                    )

                    generate_btn = gr.Button(
                        "🎨 Generate Image",
                        variant="primary",
                        size="lg"
                    )

                    # Quick presets
                    gr.Markdown("### 🚀 Quick Presets")
                    with gr.Row():
                        fast_btn = gr.Button("⚡ Ultra Fast (8 steps)", size="sm")
                        balanced_btn = gr.Button("⚖️ Balanced (15 steps)", size="sm")
                        quality_btn = gr.Button("🎨 High Quality (25 steps)", size="sm")

                with gr.Column(scale=1):
                    gr.Markdown("### 🖼️ Generated Image")
                    output_image = gr.Image(label="Generated Image", type="pil", height=400)
                    generation_status = gr.Textbox(label="📊 Generation Status", interactive=False)

                    # Time estimation display
                    gr.Markdown("### ⏱️ Expected Generation Times")
                    gr.HTML("""
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                                padding: 15px; border-radius: 10px; color: white;">
                        <b>Colab Performance (GPU):</b><br>
                        • 8 steps: ~10-15 seconds<br>
                        • 15 steps: ~20-30 seconds<br>
                        • 25 steps: ~40-60 seconds<br>
                        <small><i>512x512 resolution</i></small>
                    </div>
                    """)

        with gr.Tab("📊 System Info"):
            with gr.Column():
                gr.Markdown("### 💻 System Information")
                memory_info = gr.Textbox(label="Memory Usage", interactive=False)
                refresh_memory_btn = gr.Button("🔄 Refresh Memory Info")

                gr.Markdown("### ⚡ Optimization Tips")
                gr.HTML("""
                <div style="background: #f0f8ff; padding: 20px; border-radius: 10px; border-left: 4px solid #4CAF50;">
                    <h4>🚀 For Fastest Generation:</h4>
                    <ul>
                        <li><b>Use 8-15 inference steps</b> (sweet spot for speed/quality)</li>
                        <li><b>Keep resolution at 512x512</b> (perfect for most uses)</li>
                        <li><b>Use runwayml/stable-diffusion-v1-5</b> (fastest model)</li>
                        <li><b>Enable GPU in Colab</b> (Runtime → Change runtime type → GPU)</li>
                        <li><b>Clear memory between generations</b> (automatic in this interface)</li>
                    </ul>

                    <h4>🎨 For Best Quality:</h4>
                    <ul>
                        <li><b>Use detailed, descriptive prompts</b></li>
                        <li><b>Add style keywords:</b> "highly detailed", "cinematic", "4k"</li>
                        <li><b>Use negative prompts</b> to avoid unwanted elements</li>
                        <li><b>Guidance scale 7-8</b> works best for most images</li>
                    </ul>
                </div>
                """)

        # Event handlers
        load_btn.click(
            fn=generator.load_model,
            inputs=[model_dropdown],
            outputs=[load_status],
            show_progress=True
        )

        generate_btn.click(
            fn=generator.generate_image,
            inputs=[prompt, negative_prompt, steps, guidance, width, height, seed],
            outputs=[output_image, generation_status],
            show_progress=True
        )

        # Quick preset handlers
        def set_ultra_fast():
            return 8, 7.0, 512, 512

        def set_balanced():
            return 15, 7.5, 512, 512

        def set_high_quality():
            return 25, 8.0, 640, 640

        fast_btn.click(
            fn=set_ultra_fast,
            outputs=[steps, guidance, width, height]
        )

        balanced_btn.click(
            fn=set_balanced,
            outputs=[steps, guidance, width, height]
        )

        quality_btn.click(
            fn=set_high_quality,
            outputs=[steps, guidance, width, height]
        )

        refresh_memory_btn.click(
            fn=generator.get_memory_info,
            outputs=[memory_info]
        )

        # Load initial memory info
        interface.load(
            fn=generator.get_memory_info,
            outputs=[memory_info]
        )

    return interface

# Optimized installation instructions for Colab
def print_installation_instructions():
    print("="*70)
    print("🚀 FAST SETUP FOR GOOGLE COLAB")
    print("="*70)
    print()
    print("📋 Copy and run these commands in separate Colab cells:")
    print()
    print("# ⚙️ Cell 1: Install dependencies (run once)")
    print("!pip install -q diffusers[torch] transformers accelerate")
    print("!pip install -q gradio safetensors")
    print()
    print("# 🖥️ Cell 2: Enable GPU and check status")
    print("import torch")
    print("print('GPU Available:', torch.cuda.is_available())")
    print("if torch.cuda.is_available():")
    print("    print('GPU Model:', torch.cuda.get_device_name())")
    print("    print('GPU Memory:', f'{torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB')")
    print("else:")
    print("    print('⚠️ GPU not enabled! Go to Runtime > Change runtime type > Hardware accelerator > GPU')")
    print()
    print("# 🎨 Cell 3: Run the generator")
    print("# (paste the main script here)")
    print()
    print("⏱️ Expected times with optimizations:")
    print("   • Model loading: 1-2 minutes (first time only)")
    print("   • Image generation: 10-60 seconds (depending on settings)")
    print("="*70)

if __name__ == "__main__":
    print_installation_instructions()

    # Create and launch the interface
    interface = create_interface()

    # Launch with optimized settings for Colab
    interface.launch(
        share=True,  # Creates public link
        inbrowser=True,  # Opens in browser
        show_error=True,  # Show errors in interface
        quiet=False  # Show progress in console
    )